# Pandas
> kaggle course https://www.kaggle.com/learn/pandas

## Creating, Reading and Writing


In [ ]:
import pandas as pd

# There are two core objects in pandas: the DataFrame and the Series.
# A DataFrame is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.
pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]}) # 2 rows 2 columns
fruits = pd.DataFrame([[30, 21]], columns=['Apples', 'Bananas']) # 1 row 2 columns

# The dictionary-list constructor assigns values to the column labels, but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the row labels. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])
# A Series, by contrast, is a sequence of data values. If a DataFrame is a table, a Series is a list. And in fact you can create one with nothing more than a list:
pd.Series([1, 2, 3, 4, 5])
# A Series is, in essence, a single column of a DataFrame. So you can assign row labels to the Series the same way as before, using an index parameter. However, a Series does not have a column name, it only has one overall name:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')
# The Series and the DataFrame are intimately related. It's helpful to think of a DataFrame as actually being just a bunch of Series "glued together".

# Reading data files
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv")
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
# We can use the shape attribute to check how large the resulting DataFrame is:
wine_reviews.shape
# We can examine the contents of the resultant DataFrame using the head() command, which grabs the first five rows:
wine_reviews.head()
# Save DataFrame to csv
wine_reviews.to_csv('wine_reviews.csv')


## Indexing, Selecting & Assigning

In [ ]:
# Native accessors
# Native Python objects provide good ways of indexing data. Pandas carries all of these over, which helps make it easy to start with.
reviews = wine_reviews
reviews['country'][0]

# Indexing in pandas
# The indexing operator and attribute selection are nice because they work just like they do in the rest of the Python ecosystem. As a novice, this makes them easy to pick up and use. However, pandas has its own accessor operators, loc and iloc. For more advanced operations, these are the ones you're supposed to be using.
# Both loc and iloc are row-first, column-second.

# To select the first row of data in a DataFrame, we may use the following:
reviews.iloc[0] # return the first row
# To get a column with iloc, we can do the following:
reviews.iloc[:, 0]
# The second paradigm for attribute selection is the one followed by the loc operator: label-based selection. In this paradigm, it's the data index value, not its position, which matters.
# For example, to get the first entry in reviews, we would now do the following:
reviews.loc[0, 'country'] # return one cell: row 0 column country

cols = ['country', 'variety']
df = reviews.loc[:99, cols]
# or
cols_idx = [0, 11]
df = reviews.iloc[:100, cols_idx]

# When choosing or transitioning between loc and iloc, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.
# iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

# Manipulating the index
reviews.set_index("title")

# Conditional selection
reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)]
reviews.loc[reviews.country.isin(['Italy', 'France'])]
reviews.loc[reviews.price.notnull()]

# Assigning data
reviews['critic'] = 'everyone'
reviews['index_backwards'] = range(len(reviews), 0, -1)

## Summary Functions and Maps

In [ ]:
# Summary functions
# Pandas provides many simple "summary functions" (not an official name) which restructure the data in some useful way. 
reviews.points.describe()
reviews.points.mean()
reviews.taster_name.unique()
reviews.taster_name.value_counts()

# Maps
# A map is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. In data science we often have a need for creating new representations from existing data, or for transforming data from the format it is in now to the format that we want it to be in later.

# suppose that we wanted to remean the scores the wines received to 0
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)
# apply() is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.
def remean_points(row):
    row.points = row.points - review_points_mean
    return row
reviews.apply(remean_points, axis='columns')
# Note that map() and apply() return new, transformed Series and DataFrames, respectively. They don't modify the original data they're called on.

# Pandas provides many common mapping operations as built-ins. For example, here's a faster way of remeaning our points column:
review_points_mean = reviews.points.mean()
reviews.points - review_points_mean

## Grouping and Sorting

In [ ]:
# groupby() created a group of reviews which allotted the same point values to the given wines. Then, for each of these groups, we grabbed the points() column and counted how many times it appeared.
reviews.groupby('points').points.count()
reviews.groupby('points').price.min()
# here's how we would pick out the best wine by country and province:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])
reviews.groupby(['country']).price.agg([len, min, max])
# A multi-index differs from a regular index in that it has multiple levels.
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
# converting back to a regular index
countries_reviewed.reset_index()

countries_reviewed.sort_values(by='len', ascending=False)
countries_reviewed.sort_index()
countries_reviewed.sort_values(by=['country', 'len'])

## Data Types and Missing Values

In [ ]:
# The data type for a column in a DataFrame or a Series is known as the dtype.
reviews.price.dtype
# convert a column of one type into another
reviews.points.astype('float64')
# To select NaN entries
reviews[pd.isnull(reviews.country)]
reviews.region_2.fillna("Unknown")
reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

## Renaming and Combining


In [ ]:
# rename(), which lets you change index names and/or column names.
reviews.rename(columns={'points': 'score'})
reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})
reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')
# When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series
# The simplest combining method is concat(). Given a list of elements, this function will smush those elements together along an axis.
canadian_youtube = pd.read_csv("../input/youtube-new/CAvideos.csv")
british_youtube = pd.read_csv("../input/youtube-new/GBvideos.csv")
pd.concat([canadian_youtube, british_youtube])
# join() lets you combine different DataFrame objects which have an index in common.
left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])
left.join(right, lsuffix='_CAN', rsuffix='_UK')